<p style='text-align:center'>
    <strong>This notebook documents implementation of the K-Nearest Neighbors (KNN) algorithm project.</strong>
</p>

# **K-Nearest Neighbors (KNN) Classification - Portfolio Project**

As a data science enthusiast, I wanted to explore how KNN can be applied to real-world classification problems. This project simulates a scenario where I'm analyzing patient data to detect patterns that could improve diagnostic accuracy. This hands-on work helps strengthen my practical understanding of machine learning workflows.

## Background

The K-Nearest Neighbors (KNN) algorithm is a simple, yet effective, method used for both classification and regression. In this project, I focus on using KNN for classification tasks as an exploratory step before moving on to more complex models.

## Objectives

Through this project, my goals are to:
- Understand how the KNN algorithm works.
- Apply preprocessing techniques such as scaling and normalization.
- Train a KNN classifier using real-world-like data.
- Evaluate model performance and understand its limitations.
- Visualize decision boundaries and interpret the results.

----

## Setup

Let's begin by importing the required libraries for this project.

In this project, I use the following Python libraries:

*   `pandas` for data handling
*   `numpy` for numerical operations
*   `matplotlib` and `seaborn` for data visualization
*   `sklearn` for machine learning and evaluation tools

### Installing Required Libraries

If you're running this on a fresh environment, use the following command to install missing libraries.

Some libraries used here may not be pre-installed in your environment. Run the following cell to install any missing packages.

In [1]:
%%capture
%pip install numpy==1.26.4 pandas==2.2.2 matplotlib==3.8.4 seaborn==0.13.2 scikit-learn==1.5.0 xlrd==2.0.1 openpyxl==3.1.4

### Importing required libraries

_We recommend that you import all required libraries in one place (here):_


In [2]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from itertools import accumulate
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, cohen_kappa_score, confusion_matrix
from sklearn.feature_selection import f_classif
from sklearn.utils import resample

# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

sns.set_context('notebook')
sns.set_style('white')

## Load the data

The data set that you'll use is about classifying patients into diabetes positive or negative given their medical information such as their cholesterol, glucose levels, age, gender, height, waist, and hip measurements. This data set was taken from: [data.world](https://data.world/search?q=diabetes+classification).

Start by loading the data into a `pandas.DataFrame`.


In [3]:
df = pd.read_excel('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/X4i8vXLw81g4wEH473zIFA/Diabetes-Classification.xlsx')

Take a look at some sample rows from the data set that you loaded.


In [4]:
df.head()

,Patient number,Cholesterol,Glucose,HDL Chol,Chol/HDL ratio,Age,Gender,Height,Weight,BMI,Systolic BP,Diastolic BP,waist,hip,Waist/hip ratio,Diabetes,Unnamed: 16,Unnamed: 17
0,1,193,77,49,3.9,19,female,61,119,22.5,118,70,32,38,0.84,No diabetes,6.0,6.0
1,2,146,79,41,3.6,19,female,60,135,26.4,108,58,33,40,0.83,No diabetes,NaN,NaN
2,3,217,75,54,4.0,20,female,67,187,29.3,110,72,40,45,0.89,No diabetes,NaN,NaN
3,4,226,97,70,3.2,20,female,64,114,19.6,122,64,31,39,0.79,No diabetes,NaN,NaN
4,5,164,91,67,2.4,20,female,70,141,20.2,122,86,32,39,0.82,No diabetes,NaN,NaN


The last two columns are not relevant. You can remove them.


In [5]:
df.drop(columns=['Unnamed: 16', 'Unnamed: 17'])

,Patient number,Cholesterol,Glucose,HDL Chol,Chol/HDL ratio,Age,Gender,Height,Weight,BMI,Systolic BP,Diastolic BP,waist,hip,Waist/hip ratio,Diabetes
0,1,193,77,49,3.9,19,female,61,119,22.5,118,70,32,38,0.84,No diabetes
1,2,146,79,41,3.6,19,female,60,135,26.4,108,58,33,40,0.83,No diabetes
2,3,217,75,54,4.0,20,female,67,187,29.3,110,72,40,45,0.89,No diabetes
3,4,226,97,70,3.2,20,female,64,114,19.6,122,64,31,39,0.79,No diabetes
4,5,164,91,67,2.4,20,female,70,141,20.2,122,86,32,39,0.82,No diabetes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,386,227,105,44,5.2,83,female,59,125,25.2,150,90,35,40,0.88,No diabetes
386,387,226,279,52,4.3,84,female,60,192,37.5,144,88,41,48,0.85,Diabetes
387,388,301,90,118,2.6,89,female,61,115,21.7,218,90,31,41,0.76,No diabetes
388,389,232,184,114,2.0,91,female,61,127,24.0,170,82,35,38,0.92,Diabetes


Before you start analyzing anything, produce a frequency table to see what proportion that you have of patients with and without diabetes.


In [6]:
frequency_table = df['Diabetes'].value_counts()
props = frequency_table.apply(lambda x: x / len(df['Diabetes']))
print(props)

Diabetes
No diabetes    0.846154
Diabetes       0.153846
Name: count, dtype: float64


You reduce your data frame by keeping only the columns on which you would like to perform your classification.

You also scale your data with the `StandardScaler()` to reduce the impact of extreme values, if any.


In [7]:
df_reduced = df[["Diabetes", "Cholesterol", "Glucose", "BMI", "Waist/hip ratio", "HDL Chol", "Chol/HDL ratio", "Systolic BP", "Diastolic BP", "Weight"]]

numerical_columns = df_reduced.iloc[:, 1:10]

# Applying scaling
scaler = StandardScaler()
preproc_reduced = scaler.fit(numerical_columns)

df_standardized = preproc_reduced.transform(numerical_columns)

# Converting the standardized array back to DataFrame
df_standardized = pd.DataFrame(df_standardized, columns=numerical_columns.columns)

Now, this is how your data looks.


In [8]:
df_standardized.describe()

,Cholesterol,Glucose,BMI,Waist/hip ratio,HDL Chol,Chol/HDL ratio,Systolic BP,Diastolic BP,Weight
count,3.900000e+02,3.900000e+02,3.900000e+02,3.900000e+02,3.900000e+02,3.900000e+02,3.900000e+02,3.900000e+02,3.900000e+02
mean,7.287618e-17,-1.457524e-16,2.277381e-17,-6.741046e-16,4.327023e-17,-6.376666e-17,2.915047e-16,-3.006142e-16,-1.867452e-16
std,1.001285e+00,1.001285e+00,1.001285e+00,1.001285e+00,1.001285e+00,1.001285e+00,1.001285e+00,1.001285e+00,1.001285e+00
min,-2.896986e+00,-1.104399e+00,-2.059272e+00,-2.754229e+00,-2.217470e+00,-1.743891e+00,-2.064517e+00,-2.617764e+00,-1.942901e+00
25%,-6.328534e-01,-4.902078e-01,-7.092421e-01,-7.027598e-01,-7.108267e-01,-7.637287e-01,-6.628646e-01,-6.149262e-01,-6.729533e-01
50%,-9.484179e-02,-3.227011e-01,-1.479938e-01,-1.893664e-02,-2.472441e-01,-1.871623e-01,-4.964184e-02,-9.567210e-02,-1.092203e-01
75%,4.880041e-01,7.659498e-03,5.308134e-01,6.648866e-01,5.060777e-01,5.047173e-01,4.759777e-01,4.977612e-01,5.598254e-01
max,5.285274e+00,5.167799e+00,4.099291e+00,3.536944e+00,4.040895e+00,8.518990e+00,4.943744e+00,3.019853e+00,3.657259e+00


You'll put everything in a data frame called `df_stize` and also have the labelled target (Diabetes).


In [9]:
df_stdize = pd.concat([df_reduced['Diabetes'], df_standardized], axis=1)
df_stdize

,Diabetes,Cholesterol,Glucose,BMI,Waist/hip ratio,HDL Chol,Chol/HDL ratio,Systolic BP,Diastolic BP,Weight
0,No diabetes,-0.319013,-0.564655,-0.951944,-0.565995,-0.073401,-0.360132,-0.838071,-0.985822,-1.447312
1,No diabetes,-1.372619,-0.527432,-0.360358,-0.702760,-0.536983,-0.533102,-1.276087,-1.875972,-1.050840
2,No diabetes,0.218998,-0.601879,0.079539,0.117828,0.216339,-0.302476,-1.188484,-0.837464,0.237692
3,No diabetes,0.420753,-0.192418,-1.391841,-1.249818,1.143504,-0.763729,-0.662865,-1.430897,-1.571209
4,No diabetes,-0.969111,-0.304089,-1.300828,-0.839524,0.969660,-1.224982,-0.662865,0.201045,-0.902163
...,...,...,...,...,...,...,...,...,...,...
385,No diabetes,0.443170,-0.043523,-0.542385,-0.018937,-0.363140,0.389404,0.563581,0.497761,-1.298635
386,Diabetes,0.420753,3.194941,1.323387,-0.429231,0.100443,-0.129506,0.300771,0.349403,0.361590
387,No diabetes,2.102039,-0.322701,-1.073295,-1.660112,3.924999,-1.109668,3.542092,0.497761,-1.546430
388,Diabetes,0.555256,1.426814,-0.724411,0.528122,3.693208,-1.455608,1.439613,-0.095672,-1.249076


## Split the data set


Here, your X is all of the features for the classification, which is all of the columns except the `Diabetes` column. Alternatively, your target y is the `Diabetes` column.


In [10]:
X = df_stdize.drop(columns=['Diabetes'])
y = df_stdize['Diabetes']

# Split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

You encode those label into 0 and 1 because KNN requires numerical input. 


In [11]:
# Initialize LabelEncoder
label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.fit_transform(y_test)

## Fit the KNN model


In [12]:
# Create a KNN classifier
knn = KNeighborsClassifier()

knn.fit(X_train, y_train_encoded)

#calculate overall accuracy
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test_encoded, y_pred)
print(f'Accuracy: {accuracy:.2%}')

Accuracy: 88.46%


## Hyperparameter tuning


Let's see if we can increase the accuracy with hyperparameter tuning. Hyperparameter tuning is the process of optimizing the hyperparameters of a machine learning model to improve its performance. In this case, an example of hyperparameter would be the number of neighbors. 


In [13]:
# Create a KNN classifier
knn = KNeighborsClassifier()

param_grid = {'n_neighbors': range(1, 12)}

# Perform grid search with cross-validation
grid_search = GridSearchCV(knn, param_grid, cv=10)
grid_search.fit(X_train, y_train_encoded)


# Best parameters and best score
print("Best parameters found: ", grid_search.best_params_)
print(f"Best accuracy score: , {grid_search.best_score_:.3f}")

# Full results
results = grid_search.cv_results_
for mean_score, std_score, params in zip(results['mean_test_score'], results['std_test_score'], results['params']):
    print(f"Mean accuracy: {mean_score:.3f} (std: {std_score:.3f}) with: {params}")

Best parameters found:  {'n_neighbors': 7}
Best accuracy score: , 0.917
Mean accuracy: 0.875 (std: 0.053) with: {'n_neighbors': 1}
Mean accuracy: 0.820 (std: 0.047) with: {'n_neighbors': 2}
Mean accuracy: 0.901 (std: 0.037) with: {'n_neighbors': 3}
Mean accuracy: 0.897 (std: 0.038) with: {'n_neighbors': 4}
Mean accuracy: 0.913 (std: 0.038) with: {'n_neighbors': 5}
Mean accuracy: 0.913 (std: 0.038) with: {'n_neighbors': 6}
Mean accuracy: 0.917 (std: 0.043) with: {'n_neighbors': 7}
Mean accuracy: 0.917 (std: 0.043) with: {'n_neighbors': 8}
Mean accuracy: 0.917 (std: 0.036) with: {'n_neighbors': 9}
Mean accuracy: 0.917 (std: 0.036) with: {'n_neighbors': 10}
Mean accuracy: 0.913 (std: 0.038) with: {'n_neighbors': 11}


The best parameters found is when k = 7, and the best accuracy score is 0.917.


Currently, you are fitting the data on all of the columns. Let's see if you can have another model with less data with potentially a better accuracy.


## ANOVA for feature selection


In this section, you'll see which features are the most important when classifying patients. ANOVA, which stands for Analysis of Variance, is a statistical method that is used to compare means across multiple groups to determine if there are any statistically significant differences between the means of these groups (glucose, cholesterol, weight, BMI, and so on). It assumes that there are no differences between the group means, and results from the hypothesis testing will tell you whether or a specific feature is significant.


In [14]:
fs_score, fs_p_value = f_classif(X, y)

# Combine scores with feature names
fs_scores = pd.DataFrame({'Feature': X.columns, 'F-Score': fs_score, 'P-Value': fs_p_value})
fs_scores = fs_scores.sort_values(by='F-Score', ascending=False)

print(fs_scores)

           Feature     F-Score       P-Value
1          Glucose  350.809177  3.205119e-56
5   Chol/HDL ratio   31.242678  4.298115e-08
0      Cholesterol   16.893380  4.827353e-05
6      Systolic BP   15.931795  7.853024e-05
3  Waist/hip ratio   12.348083  4.935038e-04
8           Weight   10.588454  1.237749e-03
2              BMI    8.365055  4.040512e-03
4         HDL Chol    5.973355  1.496812e-02
7     Diastolic BP    0.947292  3.310160e-01


You see that 'Glucose' is the most important feature in your data set for predicting diabetes because its p-value is the smallest (and its F-Score is the highest). You'll do two things here to improve your performance:

1. You'll balance your data set so that you have equal numbers of records with and without diabetes. 

2. You'll see how you do when you use **just** the Glucose readings to calculate the distance to your 'neighbors' in KNN.


## Downsampling


Begin by downsampling your data set as well as having the same number of rows for diabetes positive and negative patients.


In [15]:
# Converting Diabetes column into binary (0 for No Diabetes and 1 for Diabetes)
df_stdize['Diabetes'] = np.where(df_stdize['Diabetes'] == 'Diabetes', 1, 0)
df_stdize

,Diabetes,Cholesterol,Glucose,BMI,Waist/hip ratio,HDL Chol,Chol/HDL ratio,Systolic BP,Diastolic BP,Weight
0,0,-0.319013,-0.564655,-0.951944,-0.565995,-0.073401,-0.360132,-0.838071,-0.985822,-1.447312
1,0,-1.372619,-0.527432,-0.360358,-0.702760,-0.536983,-0.533102,-1.276087,-1.875972,-1.050840
2,0,0.218998,-0.601879,0.079539,0.117828,0.216339,-0.302476,-1.188484,-0.837464,0.237692
3,0,0.420753,-0.192418,-1.391841,-1.249818,1.143504,-0.763729,-0.662865,-1.430897,-1.571209
4,0,-0.969111,-0.304089,-1.300828,-0.839524,0.969660,-1.224982,-0.662865,0.201045,-0.902163
...,...,...,...,...,...,...,...,...,...,...
385,0,0.443170,-0.043523,-0.542385,-0.018937,-0.363140,0.389404,0.563581,0.497761,-1.298635
386,1,0.420753,3.194941,1.323387,-0.429231,0.100443,-0.129506,0.300771,0.349403,0.361590
387,0,2.102039,-0.322701,-1.073295,-1.660112,3.924999,-1.109668,3.542092,0.497761,-1.546430
388,1,0.555256,1.426814,-0.724411,0.528122,3.693208,-1.455608,1.439613,-0.095672,-1.249076


In [16]:
# Number of rows for positive diabetes
positive_diabetes = df_stdize[df_stdize['Diabetes'] == 1].shape[0]
print('Number of rows for positive diabetes: ', positive_diabetes)

# Sample negative cases to match positive cases
negative_diabetes = df_stdize[df_stdize['Diabetes'] == 0]
negative_diabetes_downsampled = resample(negative_diabetes, replace=False, n_samples=positive_diabetes, random_state=42)

# Put positive and negative diabetes case into one df -> balanced
balanced = pd.concat([negative_diabetes_downsampled, df_stdize[df_stdize['Diabetes'] == 1]])
balanced.sample(5)

Number of rows for positive diabetes:  60


,Diabetes,Cholesterol,Glucose,BMI,Waist/hip ratio,HDL Chol,Chol/HDL ratio,Systolic BP,Diastolic BP,Weight
341,0,-2.896986,-0.266865,-1.543530,-0.155701,-2.217470,1.138940,-1.188484,-2.469405,-1.447312
368,1,1.048433,4.367488,0.473930,1.348710,-0.768775,1.369567,0.607383,0.275224,0.807620
275,1,0.039661,1.277919,-0.072149,0.528122,0.274286,-0.417789,-0.969476,-1.134180,-0.679148
346,0,-1.439871,4.907233,-0.618229,-0.292466,-0.247244,-0.821385,0.037961,-0.095672,-0.480912
364,0,2.214125,-0.285477,-0.239007,0.938416,0.332234,0.562374,0.125565,-0.837464,0.163354


You see that the data set is now balanced below with 60 rows per category.


In [17]:
balanced['Diabetes'].value_counts()

Diabetes
0    60
1    60
Name: count, dtype: int64

## Fitting on simpler model


In this step, you use only Glucose to classify whether a patient is diabetes positive or not.


In [18]:
X_simple = balanced[['Glucose']]
y = balanced['Diabetes']

# Split the data
X_train_simple, X_test_simple, y_train_simple, y_test_simple = train_test_split(X_simple, y, test_size=0.2, random_state=42)


In [19]:
knn_simple = KNeighborsClassifier()
knn_simple.fit(X_train_simple, y_train_simple)
y_pred_simple = knn_simple.predict(X_test_simple)
accuracy = accuracy_score(y_test_simple, y_pred_simple)
print(f'Accuracy: {accuracy:.2%}')

Accuracy: 91.67%


This time, the accuracy is 91.67%, which is good considering that you are fitting on only the Glucose column instead of the all of the columns.


---
## Evaluating KNN


In [ ]:
# Evaluate confusion matrix
cm = confusion_matrix(y_test_encoded, y_pred)

# Print confusion matrix
print("Confusion Matrix:")
print(cm)
accuracy = accuracy_score(y_test_encoded, y_pred)
print(f'Accuracy: {accuracy:.2%}')

---


## Exercises


The following exercises use a mushroom data set from Kaggle that classifies mushrooms based on their characteristics. A mushroom is edible if `class == 1` and is poisonous if `class == 0`.


In [ ]:
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/3qPv1_g8n6KvWjyLOrjXyw/mushroom-cleaned.csv')
df.sample(5)

### Exercise 1: Split the data into test and training sets


In [ ]:
# TO DO

<details>
  <summary>Click for solution</summary>
    
```python   
X = df.drop(columns=['class'])
y = df['class']

# Split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
```    
</details>


### Exercise 2: Fit a KNN model


In [ ]:
# TO DO

### Accuracy Score

Calculating the accuracy of the KNN model on test data.

### Exercise 3: From an ANOVA table, which feature has the most importance?


In [ ]:
# TO DO

<details>
  <summary>Click for solution</summary>
    
```python   
fs_score, fs_p_value = f_classif(X, y)

# Combine scores with feature names
fs_scores = pd.DataFrame({'Feature': X.columns, 'F-Score': fs_score, 'P-Value': fs_p_value})
fs_scores = fs_scores.sort_values(by='F-Score', ascending=False)

print(fs_scores)
```

From the table, stem-height is the most important feature to classify whether or not a mushroom is edible or not.
</details>


## Authors


[Lucy Xu](https://author.skills.network/instructors/lucy_xu)


## Contributors


[Ricky Shi](https://author.skills.network/instructors/ricky_shi?show=blogs&sort_by=name)

[Ashutosh Sagar](https://author.skills.network/instructors/ashutosh_sagar?show=all&sort_by=name)


Copyright © 2020 IBM Corporation. All rights reserved.
